# VacationPy
----

#### Note
* Keep an eye on your API usage. Use https://developers.google.com/maps/reporting/gmp-reporting as reference for how to monitor your usage and billing.

* Instructions have been included for each segment. You do not have to follow them exactly, but they are included to help you think through the steps.

In [20]:
# Dependencies and Setup
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import requests
import gmaps
import os
import json

# Import API key
import sys
key = "C:\\Users\\Kelly\\Desktop\\KEYS"
sys.path.append(key)
from GoogleAPI import gkey

### Store Part I results into DataFrame
* Load the csv exported in Part I to a DataFrame

In [21]:
city_df = pd.read_csv('clean_city_data.csv')
city_df

,Unnamed: 0,City ID,City Name,Cloudiness,Wind Speed,Temperature,Feels Like,Min Temp,Max Temp,Barometric Pressure,Humidity,Sea Level,Ground Level,Latitude,Longitude
0,0,2685750,Östersund,99.0,100.00,2721.0,267.45,272.04,272.15,1013.0,100.0,NaN,NaN,NaN,NaN
1,1,3831208,Qaanaaq,100.0,59.00,26381.0,258.75,263.81,263.81,1010.0,81.0,1010.0,1010.0,86.349741,82.118957
2,2,5554428,Ketchikan,1.0,60.00,27415.0,269.83,274.15,274.15,1011.0,80.0,NaN,NaN,-52.936276,16.030378
3,3,2206939,Bluff,56.0,11.62,28537.0,280.11,285.37,285.37,1025.0,80.0,NaN,NaN,-32.749052,85.423768
4,4,1525798,Balkhash,0.0,70.00,25215.0,244.26,252.15,252.15,1037.0,83.0,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
558,558,1805518,Jining,0.0,6.00,27508.0,266.40,275.08,275.08,1031.0,69.0,1031.0,1026.0,NaN,NaN
559,559,1856434,Murakami,75.0,360.00,27915.0,270.62,279.15,279.15,1013.0,87.0,NaN,NaN,NaN,NaN
560,560,2171069,Colac,100.0,5.81,29935.0,298.82,297.04,300.93,1017.0,45.0,NaN,NaN,NaN,NaN
561,561,3698608,Celendín,100.0,118.00,28458.0,284.25,284.58,284.58,1016.0,92.0,1016.0,748.0,NaN,NaN


### Humidity Heatmap
* Configure gmaps.
* Use the Lat and Lng as locations and Humidity as the weight.
* Add Heatmap layer to map.

In [22]:
gmaps.configure(api_key=gkey)
city_df = city_df.dropna()
locations = city_df[["Latitude", "Longitude"]].astype(float)

weights = city_df["Humidity"].astype(float)


In [23]:
# fig = gmaps.figure()

# heat_layer = gmaps.heatmap_layer(locations, weights, 
#                                  dissipating=False, max_intensity=100,
#                                  point_radius = 4)

# fig.add_layer(heat_layer)

# fig

### Create new DataFrame fitting weather criteria
* Narrow down the cities to fit weather conditions.
* Drop any rows will null values.

In [24]:
ideal_weather = city_df.loc[city_df['Humidity'] < 50]
ideal_weather = city_df.loc[city_df['Humidity'] > 30]
ideal_weather = city_df.loc[city_df['Feels Like'] < 1026.28]
ideal_weather = city_df.loc[city_df['Feels Like'] > 1005.76]
ideal_weather = city_df.loc[city_df['Max Temp'] < 296.5]
ideal_weather = city_df.loc[city_df['Min Temp'] > 291.5]
ideal_weather = city_df.loc[city_df['Wind Speed'] < 24]
ideal_weather = city_df.loc[city_df['Cloudiness'] > 40]
dropped_weather = ideal_weather.dropna(subset=['Humidity', 'Feels Like', 'Max Temp', 'Min Temp', 'Wind Speed', 'Cloudiness', 'Latitude', 'Longitude'])
hotel_checker = dropped_weather.sample(10)
hotel_checker

,Unnamed: 0,City ID,City Name,Cloudiness,Wind Speed,Temperature,Feels Like,Min Temp,Max Temp,Barometric Pressure,Humidity,Sea Level,Ground Level,Latitude,Longitude
321,321,1058381,Morondava,94.0,92.0,2986.0,300.95,298.60,298.60,1014.0,77.0,1014.0,1014.0,3.562910,73.520631
327,327,1528998,Yumen,99.0,85.0,25926.0,252.78,259.26,259.26,1038.0,77.0,1038.0,859.0,-51.299106,-22.872807
234,234,3418910,Upernavik,84.0,32.0,26509.0,261.04,265.09,265.09,1001.0,85.0,1001.0,1001.0,73.950017,170.348385
50,50,4034551,Faanui,100.0,106.0,29979.0,300.61,299.79,299.79,1013.0,74.0,1013.0,1005.0,-7.622731,-64.271465
171,171,1651810,Airai,100.0,278.0,29395.0,296.96,293.95,293.95,1008.0,94.0,1008.0,855.0,54.017123,53.724426
76,76,3628473,San Cristobal,100.0,83.0,29018.0,291.25,290.18,290.18,1015.0,89.0,1015.0,909.0,-68.946372,-9.250404
103,103,2022572,Khatanga,100.0,145.0,24647.0,240.38,246.47,246.47,1005.0,89.0,1005.0,1002.0,20.763353,12.615508
373,373,2381972,Adrar,69.0,78.0,28645.0,281.86,286.45,286.45,1016.0,33.0,1016.0,970.0,-78.653087,-165.965096
334,334,1337610,Thinadhoo,99.0,325.0,30117.0,302.57,301.17,301.17,1012.0,76.0,1012.0,1012.0,16.489648,172.918401
1,1,3831208,Qaanaaq,100.0,59.0,26381.0,258.75,263.81,263.81,1010.0,81.0,1010.0,1010.0,86.349741,82.118957


### Hotel Map
* Store into variable named `hotel_df`.
* Add a "Hotel Name" column to the DataFrame.
* Set parameters to search for hotels with 5000 meters.
* Hit the Google Places API for each city's coordinates.
* Store the first Hotel result into the DataFrame.
* Plot markers on top of the heatmap.

In [25]:
hotel_df = hotel_checker.reindex(columns = ["City Name", "Latitude", "Longitude"])
hotel_df["Hotel Name"] = ""
hotel_df = hotel_df.reset_index().rename(columns={"City Name":"City"})
hotel_df

,index,City,Latitude,Longitude,Hotel Name
0,321,Morondava,3.562910,73.520631,
1,327,Yumen,-51.299106,-22.872807,
2,234,Upernavik,73.950017,170.348385,
3,50,Faanui,-7.622731,-64.271465,
4,171,Airai,54.017123,53.724426,
5,76,San Cristobal,-68.946372,-9.250404,
6,103,Khatanga,20.763353,12.615508,
7,373,Adrar,-78.653087,-165.965096,
8,334,Thinadhoo,16.489648,172.918401,
9,1,Qaanaaq,86.349741,82.118957,


In [26]:
# #the latitudes and longitudes I saved from citipy were not getting along with the google api at this point, so I decided
#     #to get the place_id from the api and then get google's preferred coordinates from their own native api in order to 
#     #get to the hotel data.

# #get place_id for city
# placeid_url = "https://maps.googleapis.com/maps/api/place/findplacefromtext/json"
# # use iterrows to iterate through pandas dataframe
# for index, row in hotel_df.iterrows():
#         placeid_params = {
#         "input": row['City'],
#         "inputtype": "textquery",
#         "key": gkey,
#     }
#         ID = requests.get(placeid_url, params=placeid_params).json()
#         ID = (ID["candidates"][0]["place_id"])
#         print(ID)
    

In [27]:
# #get coordinates from place_id
# coordinate_url = "https://maps.googleapis.com/maps/api/place/details/json?"
# # use iterrows to iterate through pandas dataframe
# for index, row in hotel_df.iterrows():
#         coordinate_params = {
#         "place_id": ID,
#         "key": gkey,
#     }
# #     assemble url and make API request
#         coords = requests.get(coordinate_url, params=coordinate_params).json()
#         lat = coords['result']['geometry']['location']['lat']
#         lng = coords['result']['geometry']['location']['lng']
#         coords = f'{lat},{lng}'
#         print(coords)

In [32]:
# find the closest restaurant of each type to coordinates


placeid_url = "https://maps.googleapis.com/maps/api/place/findplacefromtext/json"
coordinate_url = "https://maps.googleapis.com/maps/api/place/details/json?"
hotel_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json?"
# use iterrows to iterate through pandas dataframe
for index, row in hotel_df.iterrows():
    
    #get place_id for city name
    placeid_params = {
        "input": row['City'],
        "inputtype": "textquery",
        "key": gkey,
    }
    
    ID = requests.get(placeid_url, params=placeid_params).json()
    ID = (ID["candidates"][0]["place_id"])
    City = row['City']
    print(f'City = {City}')
    

#     use place_id to get coordinates that agree with google's api
    coordinate_params = {
        "place_id": ID,
        "key": gkey,
    }
#     assemble url and make API request
    coords_req = requests.get(coordinate_url, params=coordinate_params).json()
    lat = coords_req['result']['geometry']['location']['lat']
    lng = coords_req['result']['geometry']['location']['lng']
    try:
        Country = coords_req['result']['address_components'][2]['long_name']
        print(f'Country = {Country}')
    except:
        print(f'County = n/a')

    
    coords = f'{lat},{lng}'
    
    hotel_params = {
        "location": coords,  
        "radius": 5000,
        "types": "lodging",
        "key": gkey,
    }
    
#     assemble url and make API request
    print(f"Retrieving Results for hotels near {City}:")
    response = requests.get(hotel_url, params=hotel_params).json()
    # extract results
    try:
        results = response['results'][0]['name']
    except:
        results = "n/a"

    
    
    try:
        print(f"Closest hotel to {City} is {results}.")
    except (KeyError, IndexError):
        print(f"Sorry, there are no hotels within 5000 meters of {City}. Maybe try someplace less remote?")
        
    print("------------")

City = Morondava
Country = Toliara Province
Retrieving Results for hotels near Morondava:
Closest hotel to Morondava is La Case Bambou.
------------
City = Yumen
Country = Gansu
Retrieving Results for hotels near Yumen:
Closest hotel to Yumen is 玉门宾馆.
------------
City = Upernavik
Country = Greenland
Retrieving Results for hotels near Upernavik:
Closest hotel to Upernavik is Gina's Guesthouse - Upernavik.
------------
City = Faanui
Country = French Polynesia
Retrieving Results for hotels near Faanui:
Closest hotel to Faanui is InterContinental Bora Bora Resort Thalasso Spa.
------------
City = Airai


IndexError: list index out of range

In [29]:
# NOTE: Do not change any of the code in this cell

# Using the template add the hotel marks to the heatmap
info_box_template = """
<dl>
<dt>Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
</dl>
"""
# Store the DataFrame Row
# NOTE: be sure to update with your DataFrame name
hotel_info = [info_box_template.format(**row) for index, row in hotel_df.iterrows()]
locations = hotel_df[["Lat", "Lng"]]

KeyError: 'Country'

In [ ]:
# Add marker layer ontop of heat map


# Display figure
